In [ ]:
import sys
import scipy as sp
import numpy as np
import xarray as xr
# import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from grid import generate_lats_lons
from paths import path_samoc, grid_file, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from paths import path_ocn_ctrl, path_ocn_rcp, path_results
from paths import file_RMASK_ocn, file_RMASK_ocn_rect, file_RMASK_ocn_low
from constants import imt, jmt, km
from xr_DataArrays import xr_DZ, xr_AREA

In [ ]:
# make file with only geometry data from file_ex_ocn
# xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['z_t', 'z_w', 'z_w_bot', 'z_w_top', 'ULAT', 'ULONG', 'TLAT', 'TLONG', 'KMT', 'KMU', 'UAREA', 'TAREA',\
#     'HU', 'HT', 'DXU', 'DXT', 'DYU', 'DYT', 'HTN', 'HTE', 'HUS', 'HUW', 'ANGLE', 'ANGLET']].to_netcdf(f'{path_results}/geometry/POP_t12_geometry.nc')

## make region mask xr dataarrays

In [ ]:
RMASK_ocn = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK
RMASK_ocn = RMASK_ocn.drop(['ULONG', 'ULAT'])
RMASK_ocn.to_netcdf(file_RMASK_ocn)

In [ ]:
RMASK_ocn_low = xr.open_dataset(file_ex_ocn_lpd, decode_times=False).REGION_MASK
RMASK_ocn_low = RMASK_ocn_low.drop(['ULONG', 'ULAT'])
# RMASK_ocn_low.to_netcdf(file_RMASK_ocn_low)

In [ ]:
RMASK_ocn_low.shape

RMASK_ocn_rect is generated in `regrid.ipynb`

## latitudes/longitudes from binary file

In [ ]:
lats,lons  = generate_lats_lons('ocn')
f, ax = plt.subplots(2, 2, sharey='row', constrained_layout=True)
ax[0,0].imshow(lats, origin='lowerleft')
ax[0,1].imshow(lons, origin='lowerleft')
ax[1,0].imshow(RMASK_ocn_low.TLAT, origin='lowerleft')
ax[1,1].imshow(RMASK_ocn_low.TLONG, origin='lowerleft')

In [ ]:
# from which latitude is the dipole grid x-direction not along parallels anymore 
plt.axhline(0)
10*RMASK_ocn_low.TLAT.std(dim='nlon').plot()
RMASK_ocn_low.TLAT.mean(dim='nlon').plot()
plt.ylim(-10,10)

## DZT/DZU
create DZT/DZU file with `xr_DZ` function, if file exists, read it

In [ ]:
DZT_ocn = xr_DZ('ocn', grid='T')

In [ ]:
dsh = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
dsl = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)


In [ ]:
DZT_ocn.sum('z_t').plot()

In [ ]:
(dsh.HU/1e2-DZT_ocn.sum('z_t')).plot()

In [ ]:
DZT = xr_DZ('ocn')
TAREA = xr_AREA('ocn')
print(f'The volument of the ocean is {((TAREA*DZT).sum().item()/1e9):.2e} km^3.')
print(f'The surface of the ocean is {(TAREA.sum()/1e6).item():.2e} km^2.')

In [ ]:
plt.imshow(DZT[:,1000,:], aspect='auto')
plt.colorbar()

In [ ]:
plt.imshow(TAREA, cmap='viridis', origin='lowerleft')
plt.colorbar()
plt.tight_layout()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.SouthPolarStereo())
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea(central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(central_latitude=-90, central_longitude=180))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(satellite_height=1e7, central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

### Atlantic + Arctic

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(central_latitude=60, central_longitude=-40))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(4,6))
ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea(central_longitude=-30, central_latitude=20))
ax.set_position((-.48,-.48,2,2))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
# plt.figure(figsize=(3, 3))
# ax = plt.axes(projection=ccrs.Stereographic())
# ax.coastlines(resolution='110m')
# ax.gridlines()

# plt.figure(figsize=(6, 3))
# ax = plt.axes(projection=ccrs.TransverseMercator(central_longitude=-30.0, central_latitude=20.0))
# ax.coastlines(resolution='110m')
# ax.gridlines()

In [ ]:
plt.figure(figsize=(5,5), constrained_layout=True)
ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea(central_longitude=-45, central_latitude=20))
ax.plot([-55, 20], [-34, -34], color='blue', transform=ccrs.PlateCarree())
ax.set_extent((-5e6, 1.5e6, 0, 1e7), crs=ccrs.LambertAzimuthalEqualArea())
ax.coastlines(resolution='110m')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,181,45))
gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])



In [ ]:
plt.figure(figsize=(4,6.5), constrained_layout=True)
ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea(central_longitude=-30, central_latitude=20))

rivers = cartopy.feature.NaturalEarthFeature(
    category='physical', name='rivers_lake_centerlines',
    scale='50m', facecolor='none', edgecolor='blue')

ax.add_feature(rivers, linewidth=1)

ax.plot([-55, 20], [ 70,  70], color='blue', transform=ccrs.PlateCarree())
ax.plot([-60,- 2], [ 45,  45], color='blue', transform=ccrs.PlateCarree())
ax.plot([-78,-15], [ 26,  26], color='blue', transform=ccrs.PlateCarree())
ax.plot([-65,-15], [ 10,  10], color='blue', transform=ccrs.PlateCarree())
ax.plot([-35, 15], [-10, -10], color='blue', transform=ccrs.PlateCarree())
ax.plot([-55, 20], [-34, -34], color='blue', transform=ccrs.PlateCarree())
ax.plot([ 20, 20], [-34, -70], color='blue', transform=ccrs.PlateCarree())
ax.plot([-68,-68], [-55, -68], color='blue', transform=ccrs.PlateCarree())
ax.plot([-167,-170],[68,  68], color='blue', transform=ccrs.PlateCarree())
ax.set_extent((-6e6, 3.5e6, -8.5e6, 1e7), crs=ccrs.LambertAzimuthalEqualArea())
ax.coastlines(resolution='110m')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,181,30))
gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
cax, kw = matplotlib.colorbar.make_axes(ax,location='bottom',pad=0.01,shrink=0.9)


In [ ]:
from maps import make_map

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
make_map(xa=np.log10(ds.KE[0,0,:,:].where(ds.REGION_MASK>0)), domain='ocn_T',
         proj='aa', cmap='magma', minv=1.5, maxv=4, label=r'log_10 of KE [cm$^2$ s$^{-2}$]')
#         proj='aa', cmap='Spectral_r', minv=-1.8, maxv=30, label=r'SST [$^\circ$C]')

In [ ]:
from xr_DataArrays import xr_DZ

In [ ]:
DZT = xr_DZ(domain='ocn')

In [ ]:
DZT

In [ ]:
DZT['TLAT'] = ds.TLAT
DZT['TLONG'] = ds.TLONG


In [ ]:
DZT.sum(dim='z_t').plot()

In [ ]:
make_map(xa=DZT.sum(dim='z_t').where(DZT[0,:,:]>0)/1e3, domain='ocn_T',
         proj='aa', cmap='cmo.deep', minv=0, maxv=6, label=r'depth [km]')
#         proj='aa', cmap='Spectral_r', minv=-1.8, maxv=30, label=r'SST [$^\circ$C]')

In [ ]:
from analysis.paths import file_ex_ocn_hires

In [ ]:
C = xr.open_dataset(file_ex_ocn_hires, decode_times=False)

In [ ]:
plt.imshow(C.KMT, origin='lowerleft')
plt.colorbar()
plt.tight_layout()

In [ ]:
from read_binary import read_binary_2D_double
from paths import file_geometry
pbc = read_binary_2D_double(file_geometry, 3600, 2400, 1)  # [lon, lat]
plt.imshow(pbc[500:1000, :500].T, origin='lowerleft')
plt.colorbar()

In [ ]:
DZT = generate_xr_DZ('ocn')
DZT_int = DZT.sum(dim='z_t')

In [ ]:
DZT_int.plot()
plt.figure()
DZT_int[:500, 500:1000].plot()

# Regions

In [ ]:
from paths import file_RMASK_ocn, file_RMASK_ocn_low, file_RMASK_ocn_rect

In [ ]:
for i in range(3):
    plt.figure()
    RMASK = xr.open_dataarray([file_RMASK_ocn, file_RMASK_ocn_low, file_RMASK_ocn_rect][i])
    plt.imshow(RMASK, cmap='tab20', origin='lowerleft', vmin=-1.1, vmax=18)
    plt.colorbar()
    plt.tight_layout()

In [ ]:
RMASK_ocn.TLAT.where(RMASK_ocn==1).plot()
RMASK_ocn.TLAT.where(RMASK_ocn==3).plot()
plt.ylim(800,850)

In [ ]:
RMASK_ocn.TLAT[810,1200]

In [ ]:
np.unique(C.REGION_MASK.values)

In [ ]:
from regions import regions
print(regions)

In [ ]:
%%time
for i, key in enumerate(regions):
    surf = xr_surf_int(D[0,:,:], TAREA.where(C.REGION_MASK==key)     )/1e6
    vol  = xr_vol_int( D       , TAREA.where(C.REGION_MASK==key), DZT)/1e9
    print(f'{i:2} {key:4} {regions[key]:16}: {surf:10.2e} km^2, {vol:10.2e} km^3')